In [ ]:
%%markdown

### Use Cases:
    - Data Scientists need access to data in your data warehouse.  BigQuery makes this extremely easy.

### Description: 
    - BigQuery provides a magic keyword ```%%bigquery``` used for executing queries to view the results or to populate a dataframe.

### Reference:
    - https://cloud.google.com/bigquery/docs/visualize-jupyter

### Clean up / Reset script:
    n/a   


In [ ]:
%%bigquery

-- Query BigQuery Data
WITH WeekdayData AS
(
SELECT FORMAT_DATE("%B", Pickup_DateTime) AS MonthName,
       FORMAT_DATE("%m", Pickup_DateTime) AS MonthNumber,
       FORMAT_DATE("%A", Pickup_DateTime) AS WeekdayName,
       SUM(taxi_trips.Total_Amount) AS Total_Amount
  FROM `${project_id}.${bigquery_taxi_dataset}.taxi_trips` AS taxi_trips
 WHERE taxi_trips.Pickup_DateTime BETWEEN '2020-01-01' AND '2020-12-31' 
   AND Payment_Type_Id IN (1,2,3,4)
 GROUP BY 1, 2, 3
)
SELECT MonthName,
       FORMAT("%'d", CAST(Sunday    AS INTEGER)) AS Sunday,
       FORMAT("%'d", CAST(Monday    AS INTEGER)) AS Monday,
       FORMAT("%'d", CAST(Tuesday   AS INTEGER)) AS Tuesday,
       FORMAT("%'d", CAST(Wednesday AS INTEGER)) AS Wednesday,
       FORMAT("%'d", CAST(Thursday  AS INTEGER)) AS Thursday,
       FORMAT("%'d", CAST(Friday    AS INTEGER)) AS Friday,
       FORMAT("%'d", CAST(Saturday  AS INTEGER)) AS Saturday,
  FROM WeekdayData
 PIVOT(SUM(Total_Amount) FOR WeekdayName IN ('Sunday','Monday','Tuesday','Wednesday','Thursday','Friday','Saturday'))
ORDER BY MonthNumber;

In [ ]:
%%bigquery myDataframe
WITH MonthlyData AS
(
SELECT FORMAT_DATE("%B", taxi_trips.Pickup_DateTime) AS MonthName,
       FORMAT_DATE("%m", taxi_trips.Pickup_DateTime) AS MonthNumber,
       CASE WHEN taxi_trips.Payment_Type_Id = 1 THEN 'Credit'
            WHEN taxi_trips.Payment_Type_Id = 2 THEN 'Cash'
            ELSE 'Other'
         END AS PaymentDescription,
       SUM(taxi_trips.Total_Amount) AS Total_Amount
  FROM `${project_id}.${bigquery_taxi_dataset}.taxi_trips` AS taxi_trips
 WHERE taxi_trips.Pickup_DateTime BETWEEN '2020-01-01' AND '2020-12-31' 
   AND Passenger_Count IS NOT NULL
   AND Payment_Type_Id IN (1,2,3,4)
 GROUP BY 1, 2, 3   
)
SELECT MonthName,
       CAST(Credit   AS INTEGER) / 1000 AS Credit,
       CAST(Cash     AS INTEGER) / 1000 AS Cash,
       CAST(Other    AS INTEGER) / 1000 AS Other
  FROM MonthlyData
 PIVOT(SUM(Total_Amount) FOR PaymentDescription IN ('Credit', 'Cash', 'Other'))
ORDER BY MonthNumber;

In [ ]:
display(myDataframe)

In [ ]:
%matplotlib inline

In [ ]:
ax = myDataframe.plot(kind="bar", x="MonthName", y=["Cash","Credit","Other"], figsize=(15, 7))
ax.set_title("Monthly Amounts by Payment Type")
ax.set_xlabel("Month")
ax.set_ylabel("Amount (thousands)")
ax.set_ylim([0, 100000])